In [ ]:
import database.manipulate_db as mdb
import database.generate_query as gquery
import pandas as pd

#cria conexão com o banco de dados
engine = mdb.create_engine_db_connection('localhost', 'root', 'root123', 'trabalho_pratico_ii')

In [ ]:
#transforma todos os dados dos arquivos csv do diretório em dados em tabelas no banco de dados MySQL
full_path = './file/' #endereço onde constam os arquivos a serem inseridos no banco de dados
file_type = 'csv' #extensão dos arquivos
to_sql_if_exists='replace' #tipo de ação ao inserir os dados no banco
result = mdb.file_to_db(engine, full_path, file_type, to_sql_if_exists) #lê arquivos de um diretório e insere no banco de dados
print(result)

In [ ]:
#busca os nomes de tabelas do banco de dados
table_schema = 'trabalho_pratico_ii' #nome do schema/banco de dados
file_path = './file/database.sql' #endereço onde será salvado o arquivo .sql com scripts do banco de dados
open_file_mode = 'a' #modo de abertura/escrita do arquivo 
df_tables = mdb.return_db_tables_name(engine, table_schema) #retorna os nomes das tabelas do banco de dados
#substitui valores no DataFrame e converte colunas em valores numéricos
new_values = {'--': '0', 'None': '', ',': '-'} #valores a serem substituídos nos arquivos
for table in df_tables['TABLE_NAME']: #lê cada tabela do banco de dados
    if 'game_logs' not in table:
        if table == 'basic_stats':
            where_clausule = 'current_status = "Active"'
        else:
            where_clausule = 'year = 2016'
        #gera arquivo com os scripts para criação do banco de dados
        gquery.generate_file_whit_db_scripts(file_path, open_file_mode, engine, where_clausule, new_values, table_schema, table)

In [ ]:
table_schema = 'trabalho_pratico_ii' #nome do schema/banco de dados
df_tables = mdb.return_db_tables_name(engine, table_schema) #retorna os nomes das tabelas do banco de dados
df = pd.DataFrame() #cria um DataFrame vazio
join_column = 'player_id' #coluna base para união dos DataFrames
for table in df_tables['TABLE_NAME']:
    table = f'{table_schema}.{table}' #adiciona o nome do schema/banco de dados no nome da tabela
    query = gquery.query_create_select(table, '*', None) #cria query select
    if df.empty: #se DataFrame vazio
        df = mdb.convert_sql_to_dataframe(query, engine) #busca dados do banco de dados e os converte em um DataFrame
    else:        
        df_table = mdb.convert_sql_to_dataframe(query, engine) #busca dados do banco de dados e os converte em um DataFrame
        df = pd.merge(df, df_table, how='left', on=join_column, suffixes=('', '_remove')) #une o DataFrame existente com o DataFrame com os dados da última tabela buscada
        df.drop([i for i in df.columns if 'remove' in i], axis=1, inplace=True) #remove do DataFrame as colunas duplicadas
    df = mdb.convert_columns_to_numeric(df, 0, 'ignore') #substitui valores nulos e converte colunas em tipo numérico
    
    #escreve df em um arquivo e salva
    file = open('df.csv', 'w') #abre/cria um arquivo e define modo de escrita 
    file.write(str(df.columns.to_list()).replace('[', '').replace(']', '') + '\n') #escreve no arquivo as colunas do DataFrame
    for row in df.values.tolist(): #lê cada linha do DataFrame transformadas em formado lista
        file.write(str(row).replace('[', '').replace(']', '') + '\n') #escreve no arquivo a linha susbtituindo caracteres e adicionando quebra de linha
    file.close() #fecha o arquivo     
df #mostra DataFrame